**Notes:** need to download the csv ('https://docs.google.com/spreadsheets/d/1j1zp8yHJbXMwSe58bW38MODuD9wOYDWfPhklsKTqjqI/edit?usp=sharing'), copy/change path and create df in step 1. Change the user agen in 'headers' to your user agent (search 'my user agent' in chrome). 

Be careful not to exceed Amazon's request rate limit! The first scrape takes aat least 30 mins!

In [ ]:
#import packages
import pandas as pd
import numpy as np
import requests
import time
import bs4
from bs4 import BeautifulSoup
import random
import re

#default mode
scraper_path = '/content/Arms_AMZN - Sheet1 (1).csv' #download the spreadsheet that directs the bot at 'https://docs.google.com/spreadsheets/d/1j1zp8yHJbXMwSe58bW38MODuD9wOYDWfPhklsKTqjqI/edit?usp=sharing', its pretty straightforward
wait_time = 5
new_releases = 'https://www.amazon.com/gp/new-releases/'
best_sellers = 'https://www.amazon.com/Best-Sellers/zgbs/ref=zg_bsnr_tab'
movers_and_shakers = 'https://www.amazon.com/gp/movers-and-shakers/ref=zg_bs_tab'
most_wished_for = 'https://www.amazon.com/gp/most-wished-for/ref=zg_bsms_tab'
gift_ideas = 'https://www.amazon.com/gp/most-gifted/ref=zg_mw_tab'

website = new_releases



print('would you like to run in default mode? (yes/no) if so, set parameters up top')
default= input()
if default == 'yes':
  print('default is on')
else:
  print('enter the path to the shared file above, after downloading+copying')
  scraper_path = input()
  print('seconds between url requests are randomly spread between 1 and an upper integer, enter your integer [longer: (10-15s) more complete, (3-5s) short: faster]')
  wait_time = int(input())
  print('which AMZN ranked category would you like to scrape? ("best sellers","new releases", "most wished for", "gift ideas"')
  category = input()
  category = str(category).lower()
  if category == 'best sellers':
    website = best_sellers
  else: 
    print('not best sellers')
  if category == 'new releases':
    website = new_releases
  else: 
    print('not new releases')
  if category == 'most wished for':
    website = most_wished_for
  else: 
    print('not most wished for')
  if category == 'gift ideas':
    website = gift_ideas
  else: 
    print('not gift ideas')
print('is this a test run? if so, enter "yes"')
try_ = input()
if try_ == 'yes':
  print('how many categories would you like to scrape? It starts at the top...')
  print('note: no point in testing >2 categories unless you would like to test >15 (most links just repeat)')
  test_iterations = int(input())
else:
  print('the full run takes a while!')
df = pd.read_csv(scraper_path)

#get page url's for all individual categories (note: URL IS VARIABLE, you should be able to use best sellers or another ranked AMZ category)
cat_names1 = {'names':[]}
cat_names2 = {'names':[]}
cat_names3 = {'names':[]}

if try_ == 'yes':
  try_or_not = test_iterations
else:
  try_or_not = len(df)

for x in range(try_or_not):
  try:
    num2 = df['second_arm'][x]
    num2 = int(num2) - 1
  except:
    print('num2 problem')
  try:
    num3 = df['third_arm'][x]
    num3 = int(num3) - 1
  except:
    print('num3 problem')
  try:
    url = website
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    cat_names = {'names':[]}
    html = requests.get(url,headers = headers).text
    soup = bs4.BeautifulSoup(html, "html.parser")
    for tag in soup.findAll("div", {"class": "_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-small__nleKL"}):
            cat_names['names'].append(tag.findNext("a"))
  except:
    print('arm 1 error')
  try:
    url = 'https://www.amazon.com'+ str(cat_names['names'][num2]).split('"')[1]
    html = requests.get(url,headers = headers).text
    soup = bs4.BeautifulSoup(html, "html.parser")
    for tag in soup.findAll("div", {"class": "_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz"}):
            cat_names1['names'].append(tag.findAll("a"))
  except:
      print('arm 2 error')
  try:
    cat_names1['names']
    url2 = 'https://www.amazon.com'+ str(cat_names1['names'][num3]).split('"')[1]
    html = requests.get(url2,headers = headers).text
    soup = bs4.BeautifulSoup(html, "html.parser")
    for tag in soup.findAll("div", {"class": "_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-large__1z5B8"}):
            cat_names2['names'].append(tag.findAll("a"))
  except:
    print('arm 3 error')
  time.sleep(random.randint(1, wait_time))

#collect all collected links, droping any duplicates
all1 = {'link':[],'name':[]}
for x in range(len(cat_names1['names'])):
  for xx in range(len(cat_names1['names'][x])):
    all1['link'].append('https://www.amazon.com'+str(cat_names1['names'][x][xx]).split('"')[1])
    all1['name'].append(str(cat_names1['names'][x][xx]).split('"')[2])
print('length of secondary links collected is '+str(len(pd.DataFrame(all1['link']).drop_duplicates())))
print('length of secondary names collected is '+str(len(pd.DataFrame(all1['name']).drop_duplicates())))

all2 = {'link':[],'name':[]}
for x in range(len(cat_names2['names'])):
  for xx in range(len(cat_names2['names'][x])):
    all2['link'].append('https://www.amazon.com'+str(cat_names2['names'][x][xx]).split('"')[1])
    all2['name'].append(str(cat_names2['names'][x][xx]).split('"')[2])
print('length of primary links collected is '+str(len(pd.DataFrame(all2['link']).drop_duplicates())))
print('length of primary names collected is '+str(len(pd.DataFrame(all2['name']).drop_duplicates())))

#mor orgnaization... getting top then bottom level urls
aaa = pd.DataFrame()
aaa['link'] = pd.DataFrame(all2['link']).drop_duplicates()[0]
aaa['name'] = pd.DataFrame(all2['name']).drop_duplicates()[0]
aa = pd.DataFrame()
aa[['link','name']] = pd.merge(pd.DataFrame(all1['link']).reset_index(),pd.DataFrame(all1['name']).reset_index(), on = 'index').drop(columns = 'index').drop_duplicates()
aa = aa.append(aaa)
aa = aa.reset_index()
aa = aa.sort_values('index').drop(columns = 'index')

cat_names = {'imageLink':[],'imageLink1':[],'brand':[],'rating':[],'ratingCount':[],'price':[]}
cat_names1 = {'imageLink':[],'imageLink1':[],'brand':[],'price':[]}
saver = {'index':[],'rating':[],'ratingCount':[]}

if try_ == 'yes':
  try_or_not = test_iterations
else:
  try_or_not = len(aa['link'])
for x in range(try_or_not):
  try:
    url = aa['link'][x]
    url2 = str(aa['link'][x]).split('ref=')[0]+'ref='+str(aa['link'][x]).split('ref=')[1].split("_1")[0]+'_2?ie=UTF8&pg=2'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    html = requests.get(url,headers = headers).text
    soup = bs4.BeautifulSoup(html, "html.parser")

    for tag in soup.findAll("div", {"class": "a-section a-spacing-mini _p13n-zg-list-grid-desktop_maskStyle_noop__3Xbw5"}):
            tt = {'imageLink':[]}
            tt['imageLink'].append(tag.findAll("img"))
            if len(tt['imageLink'])>=1:
              for x in range(len(tt['imageLink'])):
                zzz = str(tt['imageLink'][x]).split('"')[1]
                if zzz =="' class=":
                  cat_names['brand'].append(str(tt['imageLink'][x]).split('"')[2])
                else:
                  cat_names['brand'].append(zzz)
                  

                z = str(tt['imageLink'][x]).split('"')[5]
                zz = str(tt['imageLink'][x]).split('"')[7]
                if len(z)>=15:
                  cat_names['imageLink'].append(z)
                else:
                  cat_names['imageLink'].append(str(tt['imageLink'][x]).split('"')[4])
                if len(zz)>=15:                                
                  cat_names['imageLink1'].append(zz)
                else:
                  cat_names['imageLink1'].append(str(tt['imageLink'][x]).split('"')[6])
            else:
              cat_names['brand'].append(str(url))
              cat_names['imageLink1'].append(str(url))
              cat_names['imageLink1'].append(str(url))

    for tag in soup.findAll("div", {"class": "a-icon-row"}):
            x = tag.findNext("span").text
            xxx = tag.findNext("span", {"class": "a-size-small"}).text
            if len(x)>=1:
              cat_names['rating'].append(x)
            else:
              cat_names['rating'].append(str(url))
            if len(xxx)>=1:
              cat_names['ratingCount'].append(xxx)
            else:
              cat_names['ratingCount'].append(str(url))
    for tag in soup.findAll("span", {"class": "a-size-base a-color-price"}):
            x = tag.findNext("span",{"class":"_p13n-zg-list-grid-desktop_price_p13n-sc-price__3mJ9Z"}).text
            if len(x)>=1:
              cat_names['price'].append(x)
            else:
              cat_names['price'].append(str(url))
  except:
    print('first 50, error with link: '+ str(url))

  time.sleep(random.randint(1,wait_time))
#-------------------------------------------------------------------------------------------------------------
  try:
    html1 = requests.get(url2,headers = headers).text
    soup1 = bs4.BeautifulSoup(html1, "html.parser")
    for tag in soup1.findAll("div", {"class": "a-section a-spacing-mini _p13n-zg-list-grid-desktop_maskStyle_noop__3Xbw5"}):
            tt = {'imageLink':[]}
            tt['imageLink'].append(tag.findAll("img"))
            if len(tt['imageLink'])>=1:
              for x in range(len(tt['imageLink'])):
                zzz = str(tt['imageLink'][x]).split('"')[1]
                if zzz =="' class=":
                  cat_names1['brand'].append(str(tt['imageLink'][x]).split('"')[2])
                else:
                  cat_names1['brand'].append(zzz)
                z = str(tt['imageLink'][x]).split('"')[5]
                zz = str(tt['imageLink'][x]).split('"')[7]
                if len(z)>=15:
                  cat_names1['imageLink'].append(z)
                else:
                  cat_names1['imageLink'].append(str(tt['imageLink'][x]).split('"')[4])
                if len(zz)>=15:                                
                  cat_names1['imageLink1'].append(zz)
                else:
                  cat_names1['imageLink1'].append(str(tt['imageLink'][x]).split('"')[6])
            else:
              cat_names1['brand'].append(str(url2))
              cat_names1['imageLink'].append(str(url2))
              cat_names1['imageLink1'].append(str(url2))
            
    zy = 0
    for tag in soup1.findAll("div", {"class": "a-icon-row"}):
            x = tag.findNext("span",{"class":"a-icon-alt"}).text
            xxx = tag.findNext("span", {"class": "a-size-small"}).text
            if len(x)>=1:
              saver['rating'].append(x)
              saver['index'].append(zy)
            else:
              print()
            if len(xxx)>=1:
              saver['ratingCount'].append(xxx)
            else:
              print()
            zy = zy+1

    for tag in soup1.findAll("span", {"class": "a-size-base a-color-price"}):
            x = tag.findNext("span",{"class":"_p13n-zg-list-grid-desktop_price_p13n-sc-price__3mJ9Z"}).text
            if len(x)>=1:
              cat_names1['price'].append(x)
            else:
              cat_names1['price'].append(str(url2))

    print(len(cat_names1['imageLink']))
    print(len(cat_names1['imageLink1']))
    print(len(cat_names1['brand']))
    print(len(cat_names1['price']))

  except:
    print('second 50, error with link: '+ url2)
  
  time.sleep(random.randint(1,wait_time))


df = pd.DataFrame(cat_names)
df1 = pd.DataFrame(cat_names1)
df1 = df1.reset_index()
df1 = pd.merge(df1,pd.DataFrame(saver), on = 'index')

def find_number(text):
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)
df['rating_num']=df['rating'].apply(lambda x: find_number(x))
df1['rating_num']=df1['rating'].apply(lambda x: find_number(x))
df['price']=df1['price'].apply(lambda x: find_number(x))
df['price']=df1['price'].apply(lambda x: find_number(x))


listt = []
for x in range(len(df)):
  try:
    listt.append((int(df['rating_num'].iloc[x].split()[0])+int(df['rating_num'].iloc[x].split()[1])*.1)/int(df['rating_num'].iloc[x].split()[2]))
  except:
    listt.append('error')

df['rating_num'] = listt
listt = []
for x in range(len(df1)):
  try:
    listt.append((int(df1['rating_num'].iloc[x].split()[0])+int(df1['rating_num'].iloc[x].split()[1])*.1)/int(df1['rating_num'].iloc[x].split()[2]))
  except:
    listt.append('error')

df1['rating_num'] = listt

listt = []
for x in range(len(df)):
  try:
    listt.append(df['price'].iloc[x].replace('$',""))
  except:
    listt.append('error')
df['price'] = listt

listt = []
for x in range(len(df1)):
  try:
    listt.append(df1['price'].iloc[x].replace('$',""))
  except:
    listt.append('error')
df1['price'] = listt

pieces = (df1,df)
df_final = pd.concat(pieces, ignore_index = True)
df_final = df_final.drop(columns ='rating')
df_final = df_final.drop(columns = 'index')
df_final




would you like to run in default mode? (yes/no) if so, set parameters up top
yes
default is on
is this a test run? if so, enter "yes"
yes
how many categories would you like to scrape? It starts at the top...
note: no point in testing >2 categories unless you would like to test >15 (most links just repeat)
17
length of secondary links collected is 58
length of secondary names collected is 58
length of primary links collected is 20
length of primary names collected is 20
30
30
30
30
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60


In [ ]:
#backup

In [14]:
cat_names = {'imageLink':[],'imageLink1':[],'brand':[],'rating':[],'ratingCount':[],'price':[]}
cat_names1 = {'imageLink':[],'imageLink1':[],'brand':[],'rating':[],'ratingCount':[],'price':[]}

if try_ == 'on':
  try_or_not = test_iterations
else:
  try_or_not = len(aa['link'])
for x in range(try_or_not):
  try:
    url = aa['link'][x]
    url2 = str(aa['link'][x]).split('ref=')[0]+'ref='+str(aa['link'][x]).split('ref=')[1].split("_1")[0]+'_2?ie=UTF8&pg=2'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    html = requests.get(url,headers = headers).text
    soup = bs4.BeautifulSoup(html, "html.parser")
    for tag in soup.findAll("div", {"class": "a-section a-spacing-mini _p13n-zg-list-grid-desktop_maskStyle_noop__3Xbw5"}):
            tt = {'imageLink':[]}
            tt['imageLink'].append(tag.findAll("img"))
            if len(tt['imageLink'])>=1:
              for x in range(len(tt['imageLink'])):
                cat_names['brand'].append(str(tt['imageLink'][x]).split('"')[1])

                z = str(tt['imageLink'][x]).split('"')[5]
                zz = str(tt['imageLink'][x]).split('"')[7]
                if len(z)>=15:
                  cat_names['imageLink'].append(z)
                else:
                  cat_names['imageLink'].append(str(tt['imageLink'][x]).split('"')[4])
                if len(zz)>=15:                                
                  cat_names['imageLink1'].append(zz)
                else:
                  cat_names['imageLink1'].append(str(tt['imageLink'][x]).split('"')[6])
            else:
              cat_names['brand'].append(str(url))
              cat_names['imageLink1'].append(str(url))
              cat_names['imageLink1'].append(str(url))

    for tag in soup.findAll("div", {"class": "a-icon-row"}):
            x = tag.findNext("span").text
            xxx = tag.findNext("span", {"class": "a-size-small"}).text
            if len(x)>=1:
              cat_names['rating'].append(x)
            else:
              cat_names['ratingCount'].append(str(url))
            if len(xxx)>=1:
              cat_names['ratingCount'].append(xxx)
            else:
              cat_names['ratingCount'].append(str(url))
    for tag in soup.findAll("span", {"class": "a-size-base a-color-price"}):
            x = tag.findNext("span",{"class":"_p13n-zg-list-grid-desktop_price_p13n-sc-price__3mJ9Z"}).text
            if len(x)>=1:
              cat_names['price'].append(x)
            else:
              cat_names['price'].append(str(url))
  except:
    print('first 50, error with link: '+ str(url))

  time.sleep(random.randint(1,wait_time))
#-------------------------------------------------------------------------------------------------------------
  try:
    html1 = requests.get(url2,headers = headers).text
    soup1 = bs4.BeautifulSoup(html1, "html.parser")
    for tag in soup1.findAll("div", {"class": "a-section a-spacing-mini _p13n-zg-list-grid-desktop_maskStyle_noop__3Xbw5"}):
            tt = {'imageLink':[]}
            tt['imageLink'].append(tag.findAll("img"))
            if len(tt['imageLink'])>=1:
              for x in range(len(tt['imageLink'])):
                cat_names1['brand'].append(str(tt['imageLink'][x]).split('"')[1])
                z = str(tt['imageLink'][x]).split('"')[5]
                zz = str(tt['imageLink'][x]).split('"')[7]
                if len(z)>=15:
                  cat_names1['imageLink'].append(z)
                else:
                  cat_names1['imageLink'].append(str(tt['imageLink'][x]).split('"')[4])
                if len(zz)>=15:                                
                  cat_names1['imageLink1'].append(zz)
                else:
                  cat_names1['imageLink1'].append(str(tt['imageLink'][x]).split('"')[6])
            else:
              cat_names1['brand'].append(str(url2))
              cat_names1['imageLink'].append(str(url2))
              cat_names1['imageLink1'].append(str(url2))
            
    for tag in soup.findAll("div", {"class": "a-icon-row"}):
            x = tag.findNext("span").text
            xxx = tag.findNext("span").text
            if len(x)>=1:
              cat_names1['rating'].append(x)
            else:
              cat_names1['rating'].append(str(url2))
            if len(xxx)>=1:
              cat_names1['ratingCount'].append(xxx)
            else:
              cat_names1['ratingCount'].append(str(url))

    for tag in soup.findAll("span", {"class": "a-size-base a-color-price"}):
            x = tag.findNext("span",{"class":"_p13n-zg-list-grid-desktop_price_p13n-sc-price__3mJ9Z"}).text
            if len(x)>=1:
              cat_names1['price'].append(x)
            else:
              cat_names1['price'].append(str(url2))

    print(len(cat_names1['imageLink']))
    print(len(cat_names1['imageLink1']))
    print(len(cat_names1['brand']))
    print(len(cat_names1['rating']))
    print(len(cat_names1['price']))

  except:
    print('second 50, error with link: '+ url2)
  
  time.sleep(random.randint(1,wait_time))


,imageLink,imageLink1,brand,rating,ratingCount,price
0,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,Ring Chime,4.6 out of 5 stars,"52,507",$29.99
1,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,Kindle Paperwhite Fabric Cover (11th Generatio...,4.8 out of 5 stars,"4,931",$29.99
2,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,Kindle Paperwhite Fabric Cover (11th Generatio...,4.8 out of 5 stars,"4,931",$29.99
3,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,"All New, Made for Amazon, Wireless Charging Do...",4.7 out of 5 stars,"1,458",$29.99
4,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,Kindle Paperwhite Fabric Cover (11th Generatio...,4.8 out of 5 stars,"4,931",$29.99
5,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,2-Year Accident Protection Plan for Kindle Pap...,4.5 out of 5 stars,301,$24.99
6,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,2-Year Protection Plan for Fire TV Stick 4K Max,4.7 out of 5 stars,928,$5.99
7,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,NuPro Antimicrobial Screen Protector for Kindl...,4.5 out of 5 stars,433,$12.99
8,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,Kindle Paperwhite Leather Cover (11th Generati...,4.5 out of 5 stars,"3,824",$39.99
9,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,"All New, Made for Amazon, Kids Portable Charge...",4.8 out of 5 stars,218,$29.99


In [ ]:
df = pd.DataFrame(cat_names)
df1 = pd.DataFrame(cat_names1)

def find_number(text):
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)
df['rating_num']=df['rating'].apply(lambda x: find_number(x))
df1['rating_num']=df1['rating'].apply(lambda x: find_number(x))
df['price']=df1['price'].apply(lambda x: find_number(x))
df['price']=df1['price'].apply(lambda x: find_number(x))


listt = []
for x in range(len(df)):
  try:
    listt.append((int(df['rating_num'].iloc[x].split()[0])+int(df['rating_num'].iloc[x].split()[1])*.1)/int(df['rating_num'].iloc[x].split()[2]))
  except:
    listt.append('error')

df['rating_num'] = listt
listt = []
for x in range(len(df1)):
  try:
    listt.append((int(df1['rating_num'].iloc[x].split()[0])+int(df1['rating_num'].iloc[x].split()[1])*.1)/int(df1['rating_num'].iloc[x].split()[2]))
  except:
    listt.append('error')

df1['rating_num'] = listt

listt = []
for x in range(len(df)):
  try:
    listt.append(df['price'].iloc[x].replace('$',""))
  except:
    listt.append('error')
df['price'] = listt

listt = []
for x in range(len(df1)):
  try:
    listt.append(df1['price'].iloc[x].replace('$',""))
  except:
    listt.append('error')
df1['price'] = listt

pieces = (df1,df)
df_final = pd.concat(pieces, ignore_index = True)
df_final = df_final.drop(columns ='rating')
df_final



In [ ]:
len(cat_names['brand'])

10947